In [138]:
#Load file
with open ('input.txt','r',encoding='utf-8') as f:
    text = f.read()

In [139]:
len(text)

1115394

In [162]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab_size

65

In [141]:
#Tokenizer -> Mapping tokens to integers (Token = 1 character in our vocab)
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # str -> [int]
decode = lambda l: ''.join([itos[i] for i in l]) # [int] -> str

print(encode('Hello World'))
print(decode((encode('Hello World'))))


[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]
Hello World


In [142]:
#Tokenize dataset
import torch
data = torch.tensor(encode(text), dtype=torch.long)

In [143]:
#Split data -> train/validation 
n = int(len(data)*0.9)
train_data = data[:n]
val_data = data[n:]
block_size = 8

In [144]:
#Get Batch Function
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) #random batch size sample from data
    x = torch.stack([data[i:i+block_size] for i in ix]) #original
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) #target
    return x,y

xb,yb = get_batch('train_data')


In [145]:
import torch
import torch.nn as nn 
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        #idx.shape = (B,T) is the current context. Each token in idx corresponds to an index in the embedding table which stores the vector
        # representation of the token 
        logits = self.token_embedding_table(idx) #(B,T,C) => B=batch_size, T=block_size, C=vocab_size
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits, loss #logits = scores for the predicted next character in a sequence 
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx) # get preds
            logits = logits[:, -1, :] # get last time step (last token for each block in the batch) -> logits becomes (B,C)
            probs = F.softmax(logits, dim=1) #Get prob dist for next char of each block -> (B,C)
            idx_next = torch.multinomial(probs, num_samples=1) #sample from dist (B,1)
            idx = torch.cat((idx, idx_next), dim=1) #append sampled index to the running sequence (B,T+1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.4913, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [146]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [147]:
batch_size = 32
for steps in range(10000):
    xb,yb = get_batch('train')

    logits, loss = m(xb,yb) 
    optimizer.zero_grad(set_to_none=True) #clearing gradients from previous step  
    loss.backward() #calculating gradients
    optimizer.step() #using grads to update params
print(loss.item())

2.5727508068084717


In [148]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=300)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht 


In [149]:
#self attention math trick
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [101]:
# How to build context for t token in each block? Simplistic way: Sum and average the vector representations of tokens [0,t] into one feature vector for token t
# TLDR: trick = matrix multiply as weighted aggregation 
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] #(t,C)
        xbow[b,t] = torch.mean(xprev,0) #mean along 0 (rows)


In [110]:
# Showing the weighted aggregation at work with a toy example
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [113]:
# MM for weighted aggregation of past elements with Softmax for normalization 
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) #upper triangular masking so that future tokens cannot communicate with past tokens (i.e. cannot calculate weighted aggregation from future tokens as well)
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
xbow3[0]


tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [108]:
wei = wei.masked_fill(tril == 0, float('-inf'))
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [109]:
# Tells us how much of each element fuses into the position.
# But this doesn't work for self attention because token t will have different "affinitys" to the previous tokens. They aren't weighted equally
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

Self Attention
- Solves the problem of wanting to emit information from the past in a data dependent way
- How it solves:
    - Every single token at each position, will emit two vectors: Query and Key. 
    - Query Vector: What the token is looking for
    - Key Vector: What the token contains
    - We calculate affinitys by: doing a dot product between the key and the queries. My Query dot product with the keys of all the other prior tokens 
    - If key and query are aligned, dot product will be high


In [150]:
# Single head of masked self attention
torch.manual_seed(1337)
B,T,C = 4,8,32 
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B,T,16)
q = query(x) # (B,T,16)
#Transpose to get the vectors vertical for weighted aggregation 
wei = q @ k.transpose(-2,-1) # (B,T,16) @ (B,16,T) ---> (B,T,T) a T,T matrix which shows the affinities between tokens 
tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
v = value(x)
out = wei @ v # v is what gets aggregated against the weights to form the output vectors -> these vectors are the changes to the original vector embs due to context for attention

out.shape

torch.Size([4, 8, 16])

In [159]:
# x (4,8,32)
def head(x):
    head_size = 16
    key = nn.Linear(C, head_size, bias=False)
    query = nn.Linear(C, head_size, bias=False)
    value = nn.Linear(C, head_size, bias=False)
    k = key(x) # (B,T,16)
    q = query(x) # (B,T,16)
    #Transpose to get the vectors vertical for weighted aggregation 
    wei = q @ k.transpose(-2,-1) # (B,T,16) @ (B,16,T) ---> (B,T,T) a T,T matrix which shows the affinities between tokens 
    tril = torch.tril(torch.ones(T,T))
    wei = wei.masked_fill(tril == 0, float('-inf'))
    wei = F.softmax(wei, dim=1)
    v = value(x)
    out = wei @ v 

In [161]:
x.shape

torch.Size([32, 8])

In [152]:
x,y=get_batch('train')

In [160]:
attention = head(x)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x8 and 32x16)